<a href="https://colab.research.google.com/github/s-r-aman/machine_learning_playground/blob/master/fastdotai_dl/lesson5_mnistsgd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
from fastai.basics import *

In [0]:
path = Config().data_path()/'mnist'

In [0]:
!wget http://deeplearning.net/data/mnist/mnist.pkl.gz

In [11]:
!ls ../root/.fastai -a

.  ..  config.yml  data


In [0]:
!mkdir ../root/.fastai/data

In [12]:
!ls ../root/.fastai/data -a

.  ..


In [0]:
mkdir ../root/.fastai/data/mnist

In [0]:
!mv mnist.pkl.gz ../root/.fastai/data/mnist/mnist.pkl.gz

In [0]:
with gzip.open(path/'mnist.pkl.gz') as f:
  ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

In [0]:
plt.imshow(x_tain[0].rehspae(28 , 28), cmap='gray')
x_train.shape